![BAE logo](images/bae_logo.png)

# Hands-On Exercise 8.1: Using LINQ/EF

## Objective

Implement the logic to display the top 10 players listed by their best game scores in the fewest number of hands played.
    
A SQL Server database called `Tehi.mdf` has been populated with player scores. LINQ/EF will be used to provide access to this database by coupling it to a `Player` entity class.

Close the previous solution if you have not already done so.

Open the solution `Tehi` located in the directory `C:\Course\510D\Exercises\Ex081\`.

This project contains the solution files from the bonus of the previous exercise.

#### Add a connection to the Tehi player database.

If it is not already visible, from the main menu bar, select **View | Server Explorer**. This window should appear on the left side.

In the Server Explorer window, right-click **Data Connections** and select **Add Connection...**.

When the Add Connection dialog appears, click the **Change...** button to the right of the data source text box.

In the Change Data Source dialog, click `Microsoft SQL Server Database File`, and from the drop-down list, select `.NET Framework Data Provider for SQL Server`. Click **OK**.

In the Add Connection dialog click the **Browse...** button and select the file `C:\Course\510D\Data\Tehi.mdf`. Click **Open**.

Click the **Test Connection** button to make sure everything is working, then click **OK** to add the connection.

Be patient---the first time accessing the database might take 30 seconds or more.

#### Add Entity Framework packages.

The data model will be added to the desktop project. It could be placed in a separate tier (project) if desired.

Two Entity Framework packages are required to generate entities from a Microsoft SQL Server database.

- Microsoft.EntityFrameworkCore.Tools
- Microsoft.EntityFrameworkCore.SqlServer

Open the NuGet package manager console via **Tools | NuGet Package Manager | Package Manager Console** from the main menu.

<font color="red">**Make sure that **Tehi** is selected as the **Default project** in the dropdown list at the top of the console window.**</font>

Install the Microsoft.EntityFrameworkCore.Tools package using

```
Install-Package Microsoft.EntityFrameworkCore.Tools
```

Install the Microsoft.EntityFrameworkCore.SqlServer package using

```
Install-Package Microsoft.EntityFrameworkCore.SqlServer
```

#### Add Entity Framework 6 entities to access the player information.

The Entity Framework tools allow entity classes to be scaffolded (generated) automatically from the database schema. This is done using the package manager console.

Cut and paste the following command into the console.

```
Scaffold-DbContext "Data Source=(LocalDB)\MSSQLLocalDB;AttachDbFilename=C:\Course\510D\Data\Tehi.mdf;Integrated Security=True;" Microsoft.EntityFrameworkCore.SqlServer -OutputDir TehiEntities
```

The `-OutputDir` parameter specifies the project folder that will be created to hold the entity classes.

Once the entities have been scaffolded, open the **TehiEntities** folder in Solution Explorer.

There are two new classes. `Player` is an entity class representing a row of the `Player` table in the database. The `CCOURSE510DDATATEHIMDFContext` class is an Entity Framework context---basically a connection to the database.

From the main menu, select **Build | Rebuild Solution** to check everything still works.

Rename the `CCOURSE510DDATATEHIMDFContext.cs` file to `TehiEntitiesContext.cs`. Click **Yes** if prompted to rename all references at the same time.

This isn't essential, but the generated name is awful.

Review the `Player` entity class. It has three properties:

- `Name` (`string`)
- `BestHandScore` (`int`)
- `HandsDealt` (`int`)

In the Server Explorer window, expand the Data Connections tree menu to find the **Player** table, right-click it, and select **Show Table Data**. Briefly review the players in the table and notice the column names. You can see they are identifical the the names of the properties in the `Player` entity class.

Close the table view when done by clicking the **[x]** in its tab.

Now that our entity data model has been constructed by Entity Framework, we can use it as a provider so that LINQ can be used to query the database.

Review the `TehiEntitiesContext` class. There is a `#warning` pragma next to the connection string details as a reminder that this may leak important security information (e.g. passwords) and should eventually be moved to a configuration file. 

As the Tehi application is using a local database, there's no security issue here, but it's a good idea to keep connection details as configuration generally.

Connection details are often stored as environment variables, which reduces the risk of sensitive information being push to a source control repositiory.

#### Add a data accessor class to load the player data from the database.

In Solution Explorer, right-click the **Tehi** project and add a class called `DataAccessor.cs`. Ensure its definition is `public`.

In the newly created `DataAccessor` class, add as a field a `private TehiEntitiesContext` object called `db`.

Construct a new `TehiEntitiesContext` object in its declaration, as in...

```C#
private TehiEntitiesContext db = new TehiEntitiesContext();
```

Remember to add a `using TehiEntities;` statement at the top.

Add the scaffold for a get-only property to access the `Top10Players`.

It should look something like...

```C#
public List<Player> Top10Players
{
    get
    {
    }
}
```

We will first obtain all the player records unsorted. Later, we will return and modify the query to select the top 10.

In its `get` block of `Top10Players`, create a query to simply obtain all the players. Use something like the following, but replace the `???` as appropriate.

```C#
IEnumerable<???> result = from p in db.??? select p;
```

Finish with a statement that returns the result converted to a list.

You can use something like...

```C#
return result.ToList();
```

This will return all of the players in no particular order. We'll improve on this later, but let's first see what we have.

Get a clean compile.

#### Add the logic to display the players in the user interface.

In design mode, double-click the Top 10 menu and in its event method, loop through all players and display them. Use something like:

```C#
LogListBox.Items.Clear(); 

DataAccessor da = new DataAccessor(); 

foreach (Player p in da.Top10Players)
{
    LogListBox.Items.Add($"{p.Name} {p.BestHandScore} {p.HandsDealt}");
}
```

Compile and test. Select Top 10 to display the players. Do not click the Deal button by mistake. That doesn't test what we want.

All players and their scores should be listed, unsorted.

#### Modify the top-10 query to filter and sort the result.

Open `DataAccessor.cs` for editing and return to the top-10 query. Add a where clause to remove any players whose best hand score is 0; i.e., we want all players whose score is not 0.

Use something like

```C#
where p.BestHandScore != 0
```

Compile and test.

Players with scores of 0 should have been eliminated.

The "best" player is determined by who has the highest best hand achieved in the fewest number of hands dealt.

Add an `orderby` clause to the query to first sort by best hand score in descending order, then by the number of hands dealt.

An `orderby` clause can have multiple terms separated by a comma, as in `orderby <first term>, <second term>`. Remember, too, that you can use the LINQ keyword `descending` after an ordered term...

```C#
orderby p.BestHandScore descending, p.HandsDealt
```

Compile and test.

Ensure that the listing is sorted; see Bill and Charles.

Now, use the `Take(...)` query method to limit the number of players to 10. Insert it into the `return` statement before the call to `ToList()`.

It should look something like...

```C#
return result.Take(10).ToList();
```

Compile and test.

Now just the top 10 players should be shown. Your data should look something like this (probably with different values):

![Top 10 players](images/ex071_30.png)

## Congratulations! You have successfully completed the exercise. Continue to the bonus if you have more time.

# Bonus (Optional)

Use a `partial` class to add object-oriented logic to the generated `Player` entity class.

Earlier in this exercise, the player's information was displayed by composing the output using `$"{p.Name} {p.BestHandScore} {p.HandsDealt}"`.

This is not very object-oriented; providing a `ToString()` would be better. We can put it in another partial segment to the `Player` class.

#### Add a `ToString()` method to the `Player` class by using a second partial segment.

Add a new class in a file called `PlayerAdditions.cs` to the **Tehi** project's _**TechEntites** folder_. Make its declaration `public` and `partial`, and change the class name to just `Player`.

Get a clean compile using **Build | Rebuild Solution**.

Add the method:

```C#
public override string ToString()
{
    return $"{Name} {BestHandScore} {HandsDealt}";
}
```

Return to `Form1.cs`, and, in the method that displays the top 10 players, change the display line to: 

```C#
LogListBox.Items.Add(p.ToString()); 
```

Compile and test; the output should be the same.

## Congratulations! You have completed the bonus. Carry on to the _super_ bonus if you still have more time.

# Super Bonus (Optional)

Add support for tracking players by logging in and updating their best scores automatically in the database.
    
<font color="red">**This super bonus is quite long and recommended for after the course. In most cases, there will not be sufficient time to complete it during the regular course hours.**</font>

As each game is played, the best hand score and the number of hands dealt are tracked. If this score is better than the one previously recorded in the database, the database should be updated. This is very easy with LINQ/EF!

#### Add logic in the data accessor to look up and add players in the database.

Open `DataAccessor.cs` and add a method to look up a player by name. It should look like the following. Complete the `...where...` logic.

```C#
public Player? LookupPlayer(string name)
{
    IEnumerable<Player> result = from p in db.Players
                                 ...where... 
                                 select p;

    return result.FirstOrDefault();
}
```

#### Answer...

You can use this for the `where` clause...

```C#
where (p.Name.Trim().ToUpper() == name.Trim().ToUpper())
```

Next, add a method to add a new player to the database. It should have the specification `public void Add(Player player)`.

It should look like...

```C#
public void Add(Player player)
{
    db.Players.Add(player);
}
```

Add a method called `SaveChanges` to save all changes.

It should look like...
    
```C#
public void SaveChanges()
{
    db.SaveChanges();
}
```

Get a clean compile.

#### Refactor the Tehi game controller and prepare it for adding the login functionality.

Open `TehiGame.cs` for editing and find the `HandsDealt` property. Right-click on the name and select **Rename...** from the menu. Rename the property to `HandsDealtInThisGame` and click **Apply** in the dialog. All referenced uses of this property will be updated.

Following the same procedure in the previous step, rename `BestHandScore` to `BestHandScoreInThisGame`.

Get a clean compile.

#### Create a new project called `Tehi.Data` within the solution to separate data into its own tier. This is necessary to prevent circular references from `CardLib` when trying to access data.

Right-click the **Tehi** solution (not project) and add a new class library project. Call it `Tehi.Data`.

Delete the default `Class1.cs` that gets added.

Add a reference to **Tehi.Data** in both the **Tehi** and **CardLib** projects. Then add in `Form1.cs` the following using statements.

```C#
using Tehi.Data;
using Tehi.Data.TehiEntities;
```

Remove the existing `using TehiEntities;` statement.

Repeat the steps used earlier to create the Tehi entities from the database, but this time target the **Tehi.Data** project in the NuGet package manager console.

Cut the `DataAccessor.cs` and `PlayerAdditions.cs` files out of the **Tehi** project and paste them into the **Tehi.Data** project. In each of them, change the namespace to be `Tehi.Data` and `Tehi.Data.TehiEntities`, respectively.

In Solution Explorer, right-click the `TehiEntities` folder and delete it.

Right-click the **Tehi.Data** project and select Rebuild to ensure everything has been recreated properly.

Rebuild the program and run it to ensure everything is working as it did before. Check `Top10Players`.

#### Add support for enabling/disabling dealing and logging in.

To update a player's best score in the database, we need to know who they are. `TehiGame.cs` needs to support a login function. A player can no longer play without first logging in.

In the `TehiGame` class, add a `using Tehi.Data` statement.

After the `CanSwapCards` property, add three new auto-implemented properties:

- `bool CanDeal` 
- `bool CanLogin`
- `string Message` 

All should have a `private set`.

Add a constructor. Type `ctor<Tab><Tab>`, and in the body, set the `CanLogin` property to `true` and the message to `"Enter your game name and click login"```.

Get a clean compile.

Open `Form1.cs` for editing and move to the `ShowOutput()` method.

Modify the text messages reported in the list box to `"Hands in this game: "` and `"Best score in this game: "`.

Add at the last line in the `ShowOutput()` method: 

```C#
StatusLabel.Text = game.Message;
```

Move to the new game event method (`newToolStripMenuItem_Click`) and delete the line that writes to the status label.

Compile and test. The messages will not yet be correct.

Continue editing `ShowOutput ()` and, after the line that clears the list box, add 

```C#
DealButton.Enabled = game.CanDeal;
```

Move to the``FormLoad(...)` method, and add at the last line a call to `ShowOutput()`.

Compile and test. You should no longer be able to deal.

#### Add support in the Tehi game controller for logging in.

Open `TehiGame.cs` and add the following fields:

```C#
private DataAccessor tda = new DataAccessor(); 
private Player? player;
```

Move to after the `SwapCard()` method. Add the `Login(...)` method by cutting and pasting the following code.

```C#
public void Login(string name)
{
    if (name.Length < 3)
    {
        Message = "Game name must be 3 or more characters.";
        
        return;
    }
    
    CanDeal = true; 
    CanLogin = false;
    
    player = tda.LookupPlayer(name); 
    
    if (player != null)
    {
        Message = $"Player found. Overall best score is {player.BestHandScore} in {player.HandsDealt}.";

        return;
    }
  
    player = new Player(); 
    player.Name = name; 
    
    tda.Add(player); 
    
    tda.SaveChanges();
    
    Message = $"New player {name} added to database.";
}
```

Get a clean compile.

#### Add support for the login control in the view.

Open `Form1.cs` in design mode.

Double-click the **Login** button to go to its event method. Delete all existing code and add:

```C#
game.Login(NameTextBox.Text);
ShowOutput();
```

Move to the `ShowOutput()` method, and after the `DealButton` is enabled/ disabled, add:

```C#
LoginButton.Visible = game.CanLogin;
```

Remove any superfluous calls to `StatusLabel.Text = ...` in the various event methods. Leave only the one in `ShowOutput()`.

Compile and test. You should be able to log in and play the game again. Start a new game to log in again if necessary.

#### Track and update the players' all-time highest scores in the database if they get a better score in the current game.

The best all-time would be:

- Any `BestScoreInThisGame` higher than the `BestScore` recorded in the database
- Any `BestScoreInThisGame` equal to the `BestScore` in the database if the number of hands played is less

Let's add support for this.

Open `TehiGame.cs` for editing, and before the `ToString` method, add a new function called `ScoreUpdate` using the code below:

```C#
private void ScoreUpdate()
{
    if (Score > BestHandScoreInThisGame)
    {
        BestHandScoreInThisGame = Score;
    }

    if ((Score > player!.BestHandScore) ||
       ((Score == player.BestHandScore) && (HandsDealtInThisGame < player.HandsDealt)))
    {
        player.BestHandScore = Score;
        player.HandsDealt = HandsDealtInThisGame;
        tda.SaveChanges();
        Message = "New all-time best score " + Score + " in " + player.HandsDealt + " hands";
    }
    else 
    {
        Message = "...";
    }
}
```

Get a clean compile.

Move to the `Deal()` method and replace the line of code that checks if the score has improved with a call to `ScoreUpdate()`.

Repeat for the `SwapCard()` method.

Compile and test. Log in with a new game name (make one up).

After one or more hands, a message that a new all-time score has been achieved will appear.

Play for a while. See if you can get on the high-mscores list.

## Congratulations! You have completed the _super_ bonus.